In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
import os
import json
import re
import time
import datetime
from pathlib import Path

In [29]:
def adicionar_categoria(categoria, diretorio, expressao):
    if os.path.isfile('Controle\Arquivos.json'):
        with open('Controle\Arquivos.json', 'r') as controle:
            data = json.load(controle)
        if categoria not in data:
            data[categoria] = {
                "diretorio": diretorio,
                "expressao": expressao,
                "arquivos": []
            }
            if not os.path.isdir(data[categoria]["diretorio"]):
                os.makedirs(data[categoria]["diretorio"])

        with open('Controle\Arquivos.json', 'w') as controle:
            json.dump(data, controle, indent=4)

def retornar_dados():
    if os.path.isfile('Controle\Arquivos.json') == False:
        return {}

    with open('Controle\Arquivos.json', 'r') as controle:
        data = json.load(controle)
    return data

def retornar_meses_pendentes(categoria, ano):
    meses = range(1, 13)
    dados = retornar_dados()
    meses_inseridos = set(int(item['mes']) for item in dados[categoria]['arquivos'] if item['ano'] == str(ano))
    if ano == datetime.datetime.now().year:
        meses = range(1, datetime.datetime.now().month + 1)

    meses_pendentes = [mes for mes in meses if mes not in meses_inseridos]
    return meses_pendentes

def retornar_arquivos_pendentes(categorias):
    pendentes = {}
    for categoria in categorias:
        adicionar_categoria(
            categoria,
            categorias[categoria]['diretorio'] if 'diretorio' in categorias[categoria] else os.path.join(os.getcwd(), os.path.abspath('..\\'), 'Bases', categoria),
            categorias[categoria]['expressao']
        )
        pendentes[categoria] = {}
        for ano in anos:
            meses = retornar_meses_pendentes(categoria, ano)
            if meses:
                pendentes[categoria][ano] = {'meses': meses}

        if not pendentes[categoria]:
            pendentes.pop(categoria)
    return pendentes

def adicionar_arquivo(categoria, arquivo, ano, mes, tamanho, data):
    dados = retornar_dados()
    dados[categoria]['arquivos'].append({
        'arquivo': str(arquivo),
        'ano': str(ano),
        'mes': str(mes),
        'tamanho': str(tamanho),
        'data': str(data)
    })
    with open('Controle\Arquivos.json', 'w') as controle:
        json.dump(dados, controle, indent=4)



In [32]:
botoes = {
    'categoria': 'cphBody_btn',
    'ano': 'cphBody_lkAno',
    'mes': 'cphBody_lkMes',
    'exportar': 'cphBody_ExportarBOLink'
}

def abrir_navegador(link):
    opcoes = Options()
    # opcoes.add_argument('--headless') # Rodar sem abrir o navegador
    opcoes.add_argument('--auto-open-devtools-for-tabs') # Abrir o console do navegador
    opcoes.add_experimental_option('prefs', {'download.default_directory': str(Path.home() / "Downloads")})
    navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opcoes)
    navegador.set_page_load_timeout(1200)
    navegador.get(link)
    return navegador

def clicar_botao(botao, valor):
    elemento = (By.XPATH, f'//*[@id="{botoes[botao]}{valor}"]')
    antes = datetime.datetime.now()
    print(f'ANTES  | {botao} | {valor} | {antes}')
    WebDriverWait(navegador, 30).until(EC.element_to_be_clickable(elemento)).click()
    print(f'DEPOIS | {botao} | {valor} | {datetime.datetime.now() - antes} - FINALIZADO')


In [33]:
categorias = {
    'FurtoCelular': {
        'expressao': r'^DadosBO_(\d{4})_(\d{1,2})\(FURTO DE CELULAR\)\.xls$',
    },
    'RouboCelular': {
        'expressao': r'^DadosBO_(\d{4})_(\d{1,2})\(ROUBO DE CELULAR\)\.xls$'
    }
}

anos = range(2010, datetime.datetime.now().year + 1)

In [34]:
# arquivos_pendentes = retornar_arquivos_pendentes(categorias)
arquivos_pendentes = {
    'FurtoCelular': {
        2019: {
            'meses': [9]
             }}}

if arquivos_pendentes == {}:
    print('Não há arquivos pendentes para baixar')
    exit()

link = 'http://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx'
navegador = abrir_navegador(link)
for categoria in arquivos_pendentes:
    clicar_botao('categoria', categoria)
    for ano in arquivos_pendentes[categoria]:
        clicar_botao('ano', ano % 100)
        for mes in arquivos_pendentes[categoria][ano]['meses']:
            clicar_botao('mes', mes)
            clicar_botao('exportar', '')

            downloads = str(Path.home() / "Downloads")
            regex_pattern = r'^DadosBO_(\d{4})_(\d{1,2})\(FURTO DE CELULAR\)\.xls$'

            matching_files = []
            while matching_files == []:
                files = os.listdir(downloads)
                matching_files = [filename for filename in files if re.match(regex_pattern, filename)]

                if matching_files:
                    break
                print('Aguardando arquivo baixar...')
                time.sleep(5)

            print(f'Arquivos encontrado: {matching_files}')
            matching_files.sort(reverse=True)
            ultimo_arquivo_baixado = matching_files[0]
            caminho_arquivo = os.path.join(downloads, ultimo_arquivo_baixado)
            adicionar_arquivo(categoria,
                            ultimo_arquivo_baixado,
                            ano,
                            mes,
                            f'{os.path.getsize(caminho_arquivo) / 1024:.2f} KB',
                            datetime.datetime.fromtimestamp(os.path.getmtime(caminho_arquivo)))
            os.rename(caminho_arquivo, os.path.join('c:\\TCC\\Bases\\FurtoCelular', ultimo_arquivo_baixado))



ANTES  | categoria | FurtoCelular | 2023-11-02 14:46:17.107035
DEPOIS | categoria | FurtoCelular | 0:00:00.086059 - FINALIZADO
ANTES  | ano | 19 | 2023-11-02 14:46:17.193094
DEPOIS | ano | 19 | 0:00:01.955294 - FINALIZADO
ANTES  | mes | 9 | 2023-11-02 14:46:19.148388
DEPOIS | mes | 9 | 0:00:05.853377 - FINALIZADO
ANTES  | exportar |  | 2023-11-02 14:46:25.001765
DEPOIS | exportar |  | 0:07:14.300216 - FINALIZADO
Aguardando arquivo baixar...
Aguardando arquivo baixar...
Aguardando arquivo baixar...
Aguardando arquivo baixar...
Aguardando arquivo baixar...
Aguardando arquivo baixar...
Arquivos encontrado: ['DadosBO_2019_9(FURTO DE CELULAR).xls']
